# **CS224W - Final Project**

Description goes here.

# Device
You might need to use GPU for this Colab.

Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation

In [1]:
# Install torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip3 install torch-scatter -f $scatter_src
  !pip3 install torch-sparse -f $sparse_src
  !pip3 install torch-geometric
  !pip3 install -q git+https://github.com/snap-stanford/deepsnap.git
  !pip3 install -U -q PyDrive

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 31.4 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !nvcc --version
  !python -c "import torch; print(torch.version.cuda)"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
12.1


In [3]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  import torch
  print(torch.__version__)
  import torch_geometric
  print(torch_geometric.__version__)

2.1.0+cu121
2.4.0


In [4]:
# Import GraphWaveNet module
!git clone https://github.com/ggaoshen/graph_wavenet.git # NOTE: choose the right branch
%load graph_wavenet/src/graphwavenet.py
import sys
sys.path.append('graph_wavenet/src/')
from graphwavenet import GraphWaveNet
import util as util
from util import masked_rmse, masked_mse, masked_mae, masked_mape, metric

Cloning into 'graph_wavenet'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 221 (delta 33), reused 32 (delta 20), pack-reused 152
Receiving objects: 100% (221/221), 173.82 KiB | 4.24 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [5]:
!pip3 install -U torch-geometric-temporal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 25.2 MB/s eta 0:00:00
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=4ffae596d28417b183e4fa11a37ab17ff4dffc39a91baff505458626fee2db25
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
Successfully built torch-geometric-temporal
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires pyth

In [6]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context



# Temporal Datasets
# from torch_geometric_temporal.dataset import METRLADatasetLoader
# loader = METRLADatasetLoader()
# dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
# iterator = iter(dataset)
# print("METRLADatasetLoader", next(iterator))
# print("METRLADatasetLoader", next(iterator))


from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
dataset = ChickenpoxDatasetLoader().get_dataset(lags=8) # consistent with chickenpox paper
iterator = iter(dataset)
print("ChickenpoxDatasetLoader", next(iterator))
print("ChickenpoxDatasetLoader", next(iterator))


# from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
# dataset = WikiMathsDatasetLoader().get_dataset(lags=14)
# iterator = iter(dataset)
# print("WikiMathsDatasetLoader", next(iterator))
# print("WikiMathsDatasetLoader", next(iterator))


# from torch_geometric_temporal.dataset import TwitterTennisDatasetLoader
# dataset = TwitterTennisDatasetLoader().get_dataset()
# print("TwitterTennisDatasetLoader", next(iter(dataset)))


# Graph summaries
# METRLADatasetLoader Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
# ChickenpoxDatasetLoader Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])
# WikiMathsDatasetLoader Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])

ChickenpoxDatasetLoader Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])
ChickenpoxDatasetLoader Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])


In [7]:
# Run Chickenpox Dataset

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

# training
from tqdm import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() # clear cuda cache

from torch_geometric_temporal.signal import temporal_signal_split
train_ratio = 0.8
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=train_ratio)
offset = int(dataset.snapshot_count * train_ratio) # starting index for test set

in_dim = dataset[0].num_node_features # 8 treat lagged inputs as node features
out_dim = 1
num_nodes = dataset[0].num_nodes # 1068
timesteps_to_predict = 10 # 10, 20, 40 week forecast horizon
epochs = 50
lrate = 0.0001
wdecay = 0.001
save_path = "store/checkpoint"

# util.extensions_enabled controls the extensions that have been introduced to improve 
# the performance of Graph Wavenet.
util.extensions_enabled = True
print("Graph Wavenet - Extensions Enabled: {}".format(util.extensions_enabled))

model = GraphWaveNet(
    num_nodes=num_nodes,
    in_channels=in_dim,
    out_channels=out_dim,
    out_timesteps=timesteps_to_predict,
).to(device)

if not os.path.exists(save_path):
    os.makedirs(save_path)

print("start training...", flush=True)
his_loss = []
val_time = []
train_time = []
best_epoch = 0

optimizer = torch.optim.Adam(model.parameters(), lr=lrate, weight_decay=wdecay)

scheduler = None
clip = None

# As part of extensions, we enable learning rate decay and gradient clipping
if util.extensions_enabled:
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.99)
    clip = 5

model.train()

def prepare_n_period_y(dataset):

    res = []
    for data in dataset:
        res.append(data.y)
    res = torch.stack(res, dim=0)

    return res

y_all = prepare_n_period_y(dataset)
training_curve_dict = {"epoch_train_loss": []}

for epoch in tqdm(range(epochs)):
    train_loss = []
    t1 = time.time()

    for i, snapshot in enumerate(train_dataset):
        x_train = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
        if timesteps_to_predict == 1:
            y_train = snapshot.y.to(device)
        else:
            y_train = y_all[i : i + timesteps_to_predict,:].to(device)

        pred = model(x_train, snapshot.edge_index, snapshot.edge_attr).squeeze()
        loss = masked_mse(pred, y_train, 0.0) # mean squared error for loss
        loss.backward()

        if util.extensions_enabled:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        if util.extensions_enabled:
            scheduler.step()

        optimizer.zero_grad()

        train_loss.append(loss.item())

    mtrainloss = np.mean(train_loss)
    training_curve_dict['epoch_train_loss'].append(mtrainloss)
    print(f"training loss: {mtrainloss}")


# eval
model.eval()
loss = 0
for i, snapshot in enumerate(test_dataset):
    if i + timesteps_to_predict > test_dataset.snapshot_count:
        break

    x_test = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)

    if timesteps_to_predict == 1:
        y_test = snapshot.y.to(device)
    else:
        y_test = y_all[offset+i : offset+i + timesteps_to_predict,:].to(device)


    with torch.no_grad():
        pred = model(x_test, snapshot.edge_index, snapshot.edge_attr).squeeze()
    loss += masked_mse(pred, y_test, 0.0) # mean squared error as loss

loss = loss / (i+1)
loss = loss.item()
print("Test MSE Loss: {:.4f}".format(loss))


# store training loss
pd.DataFrame(training_curve_dict).to_csv("training_curve.csv")

from google.colab import files
files.download('training_curve.csv')


Graph Wavenet Extension for Performance Improvements: True
start training...


  2%|▏         | 1/50 [00:29<24:10, 29.60s/it]

training loss: 0.9671555200864266


  4%|▍         | 2/50 [00:49<19:20, 24.17s/it]

training loss: 0.9504766389182429


...

100%|██████████| 50/50 [16:23<00:00, 19.68s/it]

training loss: 0.9363971988102648


Test MSE Loss: 0.9565


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
training_curve_dict["epoch_train_loss"]

[0.9671555200864266,
 0.9504766389182429,
 0.9448828266070384,
 0.9416872223702872,
 0.9401311262669724,
 0.9390395023292157,
 0.9379883721061959,
 0.9378094548904677,
 0.937218841878561,
 0.9371071385583136,
 0.937103212842854,
 0.9365955204271326,
 0.936577372970741,
 0.9365324727509443,
 0.9370536955073476,
 0.9370811624497902,
 0.9367381829632128,
 0.9362603400038874,
 0.9369375605863042,
 0.9365482680817566,
 0.9365739453629386,
 0.9364841893406176,
 0.9369911970207241,
 0.9370590238309489,
 0.9366425331091371,
 0.9365396076222745,
 0.9369200905603243,
 0.9366607517367456,
 0.9365784009645989,
 0.9365373539942794,
 0.9366490864308505,
 0.9367937573890497,
 0.9367301905255129,
 0.936658736336522,
 0.9368471846575054,
 0.9368333900438213,
 0.9370351310410514,
 0.9363913373885359,
 0.9369788778218918,
 0.9365114796497837,
 0.9371295634475423,
 0.9365569575076423,
 0.9367384774837552,
 0.9363529833805997,
 0.9365123419409118,
 0.9367180464244107,
 0.936738853329202,
 0.936932766619252

In [9]:
!nvidia-smi

Fri Dec 15 02:44:26 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0              33W /  70W |    551MiB / 15360MiB |     16%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
# Reset GPU RAM if GPU out of memory

# !pip install numba

# from numba import cuda

# cuda.select_device(0) # choosing second GPU

# cuda.close()